In [1]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os

In [2]:
audio_train=pd.read_csv('audio_data_train.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,3,103.0,NaN,2b2,Ar,mc,LittC2SE,6.0,2.0,4.0,0.0,0.0,103_2b2_Ar_mc_LittC2SE.txt,train
1,10,105.0,NaN,1b1,Tc,sc,Meditron,8.0,8.0,0.0,0.0,0.0,105_1b1_Tc_sc_Meditron.txt,train
2,11,106.0,NaN,2b1,Pl,mc,LittC2SE,9.0,1.0,0.0,8.0,0.0,106_2b1_Pl_mc_LittC2SE.txt,train
3,12,106.0,NaN,2b1,Pr,mc,LittC2SE,9.0,2.0,7.0,0.0,0.0,106_2b1_Pr_mc_LittC2SE.txt,train


In [3]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=3000
highcut=5000

In [4]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [5]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [6]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [7]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [ ]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = [[], [], [], []]
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                classwise_cycle_list[d[1]].append(d)

0it [00:00, ?it/s]

103_2b2_Ar_mc_LittC2SE going on 0 out of 539


1it [00:00,  1.61it/s]

105_1b1_Tc_sc_Meditron going on 1 out of 539


2it [00:01,  1.56it/s]

...

258it [02:40,  1.57it/s]

158_1p3_Pr_mc_AKGC417L going on 258 out of 539


259it [02:40,  1.58it/s]

158_1p3_Tc_mc_AKGC417L going on 259 out of 539


260it [02:41,  1.59it/s]

In [ ]:
#splitting and padding cycles
def split_and_pad(original, desiredLength, sample_rate, types=0):
	if types==0:
		return split_and_pad_old(original, desiredLength, sample_rate)

	output_buffer_length = int(desiredLength*sample_rate)
	soundclip = original[0].copy()
	n_samples = len(soundclip)

	output = []
	if n_samples > output_buffer_length:
		frames = librosa.util.frame(soundclip, frame_length=output_buffer_length, hop_length=output_buffer_length//2, axis=0)
		for i in range(frames.shape[0]):
			output.append((frames[i], original[1], original[2], original[3], original[4], i, 0))

		last_id = frames.shape[0]*(output_buffer_length//2)
		last_sample = soundclip[last_id:]; pad_times = (output_buffer_length-len(last_sample))/len(last_sample)
		padded = generate_padded_samples(soundclip, last_sample, output_buffer_length, sample_rate, types)
		output.append((padded, original[1], original[2], original[3], original[4], i+1, pad_times))

	else:
		padded = generate_padded_samples(soundclip, soundclip, output_buffer_length, sample_rate, types); pad_times = (output_buffer_length-len(soundclip))/len(soundclip)
		output.append((padded, original[1], original[2], original[3], original[4], 0, pad_times))

	return output


def split_and_pad_old(original, desiredLength, sample_rate):
    output_buffer_length = int(desiredLength * sample_rate)
    soundclip = original[0].copy()
    n_samples = len(soundclip)
    total_length = n_samples / sample_rate #length of cycle in seconds
    n_slices = int(math.ceil(total_length / desiredLength)) #get the minimum number of slices needed
    samples_per_slice = n_samples // n_slices
    src_start = 0 #Staring index of the samples to copy from the original buffer
    output = [] #Holds the resultant slices
    for i in range(n_slices):
        src_end = min(src_start + samples_per_slice, n_samples)
        length = src_end - src_start
        copy = generate_padded_samples_old(soundclip[src_start:src_end], output_buffer_length)
        output.append((copy, original[1], original[2], original[3], original[4], i))
        src_start += length
    return output


def generate_padded_samples(original, source, output_length, sample_rate, types):
	copy = np.zeros(output_length, dtype=np.float32)
	src_length = len(source)
	left = output_length-src_length # amount to be padded
	# pad front or back
	prob = random.random()
	if types == 1:
		aug = original
	else:
		aug = gen_augmented(original, sample_rate)

	while len(aug) < left:
		aug = np.concatenate([aug, aug])

	if prob < 0.5:
		#pad back
		copy[left:] = source
		copy[:left] = aug[len(aug)-left:]
	else:
		#pad front
		copy[:src_length] = source[:]
		copy[src_length:] = aug[:left]

	return copy


In [ ]:
#General augmentation augmentation
def gen_augmented(original, sample_rate):
	# list of augmentors available from the nlpaug library
	augment_list = [
	#naa.CropAug(sampling_rate=sample_rate)
	naa.NoiseAug(),
	naa.SpeedAug(),
	naa.LoudnessAug(factor=(0.5, 2)),
	naa.VtlpAug(sampling_rate=sample_rate, zone=(0.0, 1.0)),
	naa.PitchAug(sampling_rate=sample_rate, factor=(-1,3))
	]
	# sample augmentation randomly
	aug_idx = random.randint(0, len(augment_list)-1)
	augmented_data = augment_list[aug_idx].augment(original)
	return augmented_data

In [ ]:
#creating sliced and padded data = audio_data
audio_data=[]
for idx, sample in enumerate(cycle_list):
            print(str(idx)+" out of "+str(len(cycle_list)))
            output = split_and_pad(sample, desired_length, sample_rate, types=1)
            audio_data.extend(output)

In [ ]:
#Device_wise_list
file_to_device = {}
device_to_id = {}
device_id = 0
files = audio_train['Filename']#os.listdir(data_dir)
device_patient_list = []
pats = []
for f in files:
    device = f.strip().split('_')[-1].split('.')[0]
    if device not in device_to_id:
        device_to_id[device] = device_id
        device_id += 1
        device_patient_list.append([])
    file_to_device[f.strip().split('.')[0]] = device_to_id[device]
    pat = f.strip().split('_')[0]
    if pat not in device_patient_list[device_to_id[device]]:
        device_patient_list[device_to_id[device]].append(pat)
    if pat not in pats:
        pats.append(pat)

In [ ]:
#creating labels for sliced and paddedd data = audio_data
class_probs=np.zeros(4)
identifiers=[]
device_wise=[]
labels=[]
for idx in range(device_id):
            device_wise.append([])
for idx, sample in enumerate(audio_data):
            class_probs[sample[1]] += 1.0
            labels.append(sample[1])
            identifiers.append(sample[2]+'_'+str(sample[3])+'_'+str(sample[1]))
            device_wise[file_to_device[sample[2]]].append(sample)

<b>audio_data</b> is the data which padded and augmented (in the simple way)

In [139]:
#creating mel spectrograms from augmented and padded data
def create_mel_raw(current_window, sample_rate, n_mels=128, f_min=50, f_max=4000, nfft=2048, hop=512, resz=1):
    S = librosa.feature.melspectrogram(y=current_window, sr=sample_rate, n_mels=n_mels, fmin=f_min, fmax=f_max, n_fft=nfft, hop_length=hop)
    S = librosa.power_to_db(S, ref=np.max)
    S = (S-S.min()) / (S.max() - S.min())
    S *= 255
    img = cv2.applyColorMap(S.astype(np.uint8), cmapy.cmap('magma'))
    height, width, _ = img.shape
    if resz > 0:
        img = cv2.resize(img, (width*resz, height*resz), interpolation=cv2.INTER_LINEAR)
    img = cv2.flip(img, 0)
    return img

In [140]:
train_dataset=[]
for index in range(len(audio_data)):
    audio=audio_data[index][0]
    # convert audio signal to spectrogram
    # spectrograms resized to 3x of original size
    audio_image = cv2.cvtColor(create_mel_raw(audio, sample_rate, f_max=f_max, 
            n_mels=n_mels, nfft=nfft, hop=hop, resz=3), cv2.COLOR_BGR2RGB)
    
    # blank region clipping
    audio_raw_gray = cv2.cvtColor(create_mel_raw(audio, sample_rate, f_max=f_max, 
            n_mels=n_mels, nfft=nfft, hop=hop), cv2.COLOR_BGR2GRAY)
    audio_raw_gray[audio_raw_gray < 10] = 0
    for row in range(audio_raw_gray.shape[0]):
        black_percent = len(np.where(audio_raw_gray[row,:]==0)[0])/len(audio_raw_gray[row,:])
        if black_percent < 0.80:
            break   
    if (row+1)*3 < audio_image.shape[0]:
        audio_image = audio_image[(row+1)*3:, :, :]
    audio_image = cv2.resize(audio_image, (audio_image.shape[1], n_mels*3), interpolation=cv2.INTER_LINEAR)
        
    
    label = audio_data[index][1]
    train_dataset.append((audio_image, label))

In [141]:
train_dataset[0][0].shape

(192, 753, 3)

In [142]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
display(X_train)
# np.save('X_train', np.concatenate(X_train, axis=0).reshape(365, 192, 753, 3))
# np.save('y_train', y_train)

# print("X_train length : "+str(len(X_train)))
# print("X_train sample length : "+str(X_train[0].shape))
# print("Array : "+str(np.concatenate(X_train, axis=0).shape))

# np.concatenate(X_train, axis=0).reshape(365, 192, 753, 3).shape

[array([[[153,  45, 127],
         [153,  45, 127],
         [103,  31,  90],
         ...,
         [ 98,  30,  91],
         [146,  43, 128],
         [146,  43, 128]],
 
        [[153,  45, 127],
         [153,  45, 127],
         [102,  31,  89],
         ...,
         [ 98,  30,  91],
         [146,  43, 128],
         [146,  43, 128]],
 
        [[153,  45, 127],
         [153,  45, 127],
         [102,  30,  88],
         ...,
         [ 98,  30,  91],
         [146,  43, 128],
         [146,  43, 128]],
 
        ...,
 
        [[149,  44, 128],
         [149,  44, 128],
         [ 99,  29,  86],
         ...,
         [ 84,  24,  87],
         [126,  36, 129],
         [126,  36, 129]],
 
        [[149,  44, 128],
         [149,  44, 128],
         [ 99,  29,  86],
         ...,
         [ 84,  24,  87],
         [126,  36, 129],
         [126,  36, 129]],
 
        [[149,  44, 128],
         [149,  44, 128],
         [ 99,  29,  86],
         ...,
         [ 84,  24,  87],
  